# TAREA 2

## INTRODUCCIÓN

El objetivo de esta tarea es comparar distintos metodos de cálculo en Python para evaluar, funcionamiento, tiempo de resolución y resultados entregados por cada uno. Para ello tomaremos como caso de estudio el cálculo del periodo de oscilación de un péndulo simple

$$T(\alpha_0) = 4 \sqrt{\frac{L}{g}} \int_{0}^{\frac{\pi}{2}} \frac{d\theta}{\sqrt{1-sen^2(\frac{\alpha_0}{2}) sen^2(\theta)}}$$

El interés estará en probar distintas funciones de las librerías de SciPy y Sympy para resolver la integral de la función. Estas funciones utilizan distintos métodos de aproximación para resolver las integrales por lo que será interesante estudiar que tan precisas son. Compararemos los resultados obtenidos tanto con valores experimentales como con valores teoricos utilizando la famosa aproximación para ángulos pequeños.

$$ T \approx 2 \pi \sqrt{\frac{L}{g}}$$

Graficaremos los resultados para facilitar la comparación y poder visualizar que tan notoria es la diferencia.


### Librerías

In [1]:
import sympy
import scipy
from scipy.integrate import quad, simpson, trapezoid, romb
from scipy import special
import time
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### Variables iniciales y valores teóricos

Aquí definimos la lista con los ángulos iniciales, se pueden agregar más para tener más puntos de comparación, el largo del pendulo, el valor teorico del periodo y dejamos expresadas las funciones para calcular los valores con los distintos métodos.

In [2]:
# Lista con los los ángulos a probar
ang = [np.pi/12, np.pi/8, np.pi/6, np.pi/4, np.pi/3, np.pi/2]

#Lista con los valores experimentales
exp = [0.14, 0.15, 0.15, 0.17, 0.18, 0.20]

#Variables del péndulo, largo y gravedad
L = 0.05
g = 9.81

#Valor teórico del periodo
T_teorico = 2*np.pi*np.sqrt(L/g)

#Definimos la constante de la integral
const = 4*np.sqrt(L/g)

#Definimos las funciones a integrar
func0 = lambda x: (const/ np.sqrt(1 - (np.sin(ang[0]/2)**2) * (np.sin(x)**2)))
func1 = lambda x: (const/ np.sqrt(1 - (np.sin(ang[1]/2)**2) * (np.sin(x)**2)))
func2 = lambda x: (const/ np.sqrt(1 - (np.sin(ang[2]/2)**2) * (np.sin(x)**2)))
func3 = lambda x: (const/ np.sqrt(1 - (np.sin(ang[3]/2)**2) * (np.sin(x)**2)))
func4 = lambda x: (const/ np.sqrt(1 - (np.sin(ang[4]/2)**2) * (np.sin(x)**2)))
func5 = lambda x: (const/ np.sqrt(1 - (np.sin(ang[5]/2)**2) * (np.sin(x)**2)))

Con las funciones y valores inciales podemos proceder a calcular la función del Periodo según el ángulo inicial, para ello, probaremos todos los métodos de integración disponibles. Una vez obtengamos los resultados los graficaremos para poder comparar cada método con respecto a los resultados experimentales y teoricos. 

### Funcion trapezoid()

El método de aproximación por trapecios es conocida por ser eficiente y dar una buena aproximación. Consiste en tomar un intervalo dado y subdividirlo en N sub-intervalos uniformes. Se toman los extremos de cada uno y se traza una recta conectandolos y formando un trapecio. Se calcula el área de cada trapecio y finalmente estas se suman para entregar el valor aproximado. 

<img src = "https://www.neurochispas.com/wp-content/uploads/2022/12/Regla-de-los-trapecios-ejercicio-3-grafica.jpg" width = "450">

In [7]:
N = 1001 #Número de puntos 
x = np.linspace(0, np.pi/2, N) #Puntos equiespaciados

#Medimos tiempo de ejecución
inicio = time.time()
trapz0 = trapezoid(func0(x), x)
trapz1 = trapezoid(func1(x), x)
trapz2 = trapezoid(func2(x), x)
trapz3 = trapezoid(func3(x), x)
trapz4 = trapezoid(func4(x), x)
trapz5 = trapezoid(func5(x), x)
fin = time.time()
tiempo_trapz = fin - inicio
print("Tiempo de ejecución en (s): ", tiempo_trapz)

#Resultados con trapecios
print("Resultados:")
print("Angulo (rad) | Periodo (s) | Error Relativo (%)")
print(f"{ang[0]:.4f}        | {trapz0:.6f}   | {abs((trapz0 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[1]:.4f}        | {trapz1:.6f}   | {abs((trapz1 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[2]:.4f}        | {trapz2:.6f}   | {abs((trapz2 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[3]:.4f}        | {trapz3:.6f}   | {abs((trapz3 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[4]:.4f}        | {trapz4:.6f}   | {abs((trapz4 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[5]:.4f}        | {trapz5:.6f}   | {abs((trapz5 - T_teorico)/T_teorico)*100:.6f}")

Tiempo de ejecución en (s):  0.0
Resultados:
Angulo (rad) | Periodo (s) | Error Relativo (%)
0.2618        | 0.450499   | 0.430058
0.3927        | 0.452932   | 0.972431
0.5236        | 0.456379   | 1.740880
0.7854        | 0.466501   | 3.997334
1.0472        | 0.481397   | 7.318201
1.5708        | 0.529466   | 18.034060


### Funcion quad()

En pocas palabras, la función quad analiza la función para ver donde esta tiene un comportamiento constante y donde esta varía mucho. Luego separa la función objetivamente en distintos intervalos no necesariamente iguales y aproxima el área bajo la curva con la forma cuadrática que mas se asimile al intervalo, ya sea rectángulos o trapezios. Entrega el valor encontrado y el error estimado.

<img src = "https://sdmntprwestcentralus.oaiusercontent.com/files/00000000-a868-61fb-a69b-3604ae3e18de/raw?se=2025-09-24T15%3A25%3A48Z&sp=r&sv=2024-08-04&sr=b&scid=aed2c9d5-51b9-585d-a4f9-346b526c8b8f&skoid=f28c0102-4d9d-4950-baf0-4a8e5f6cf9d4&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-09-23T21%3A15%3A11Z&ske=2025-09-24T21%3A15%3A11Z&sks=b&skv=2024-08-04&sig=G41Hj/q1vRH9wTWRDEca9Ve00cIH0QNY2InoAFIqaws%3D" width = "400">

In [8]:
#Medimos el tiempo de ejecución
inicio = time.time()
quad0 = quad(func0, 0, np.pi/2)
quad1 = quad(func1, 0, np.pi/2)
quad2 = quad(func2, 0, np.pi/2)
quad3 = quad(func3, 0, np.pi/2)
quad4 = quad(func4, 0, np.pi/2)
quad5 = quad(func5, 0, np.pi/2)
fin = time.time()
tiempo_quad = fin - inicio
print("Tiempo calculo en (s)", tiempo_quad)

#Mostramos resultados
print("Resultados:")
print("Angulo (rad) | Periodo (s) | Error Estimado | Error Relativo (%)")
print(f"{ang[0]:.4f}        | {quad0[0]:.6f}   | {quad0[1]:.6f}       | {abs((quad0[0] - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[1]:.4f}        | {quad1[0]:.6f}   | {quad1[1]:.6f}       | {abs((quad1[0] - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[2]:.4f}        | {quad2[0]:.6f}   | {quad2[1]:.6f}       | {abs((quad2[0] - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[3]:.4f}        | {quad3[0]:.6f}   | {quad3[1]:.6f}       | {abs((quad3[0] - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[4]:.4f}        | {quad4[0]:.6f}   | {quad4[1]:.6f}       | {abs((quad4[0] - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[5]:.4f}        | {quad5[0]:.6f}   | {quad5[1]:.6f}       | {abs((quad5[0] - T_teorico)/T_teorico)*100:.6f}")

Tiempo calculo en (s) 0.0
Resultados:
Angulo (rad) | Periodo (s) | Error Estimado | Error Relativo (%)
0.2618        | 0.450499   | 0.000000       | 0.430058
0.3927        | 0.452932   | 0.000000       | 0.972431
0.5236        | 0.456379   | 0.000000       | 1.740880
0.7854        | 0.466501   | 0.000000       | 3.997334
1.0472        | 0.481397   | 0.000000       | 7.318201
1.5708        | 0.529466   | 0.000000       | 18.034060


### Funcion Simpson

El metodo de Simpson es el siguiente, dado un intervalo [a, b] este se separa en n sub-intervalos inguales. En cada intervalo tomamos un punto medio y se obtiene un polinomio de segundo grado que pase tanto por los extremos del intervalo como por el punto medio. De esta manera, por cada intervalo vamos creando un polinomio con un área bajo la curva similar. 

<img src="https://www.lifeder.com/wp-content/uploads/2020/03/Simpson-01.jpg" width="400">




In [9]:
#Medimos el tiempo de ejecución
inicio = time.time()
N = 10001
x = np.linspace(0, np.pi/2, N)
simp0 = simpson(func0(x), x=x)
simp1 = simpson(func1(x), x=x)
simp2 = simpson(func2(x), x=x)
simp3 = simpson(func3(x), x=x)
simp4 = simpson(func4(x), x=x)
simp5 = simpson(func5(x), x=x)
fin = time.time()
tiempo_simp = fin - inicio
print("Tiempo calculo en (s)", tiempo_simp)

# Resultados 
print("Resultados:")
print("Angulo (rad) | Periodo (s) | Error Relativo (%)")
print(f"{ang[0]:.4f}        | {simp0:.6f}   | {abs((simp0 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[1]:.4f}        | {simp1:.6f}   | {abs((simp1 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[2]:.4f}        | {simp2:.6f}   | {abs((simp2 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[3]:.4f}        | {simp3:.6f}   | {abs((simp3 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[4]:.4f}        | {simp4:.6f}   | {abs((simp4 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[5]:.4f}        | {simp5:.6f}   | {abs((simp5 - T_teorico)/T_teorico)*100:.6f}")

Tiempo calculo en (s) 0.0
Resultados:
Angulo (rad) | Periodo (s) | Error Relativo (%)
0.2618        | 0.450499   | 0.430058
0.3927        | 0.452932   | 0.972431
0.5236        | 0.456379   | 1.740880
0.7854        | 0.466501   | 3.997334
1.0472        | 0.481397   | 7.318201
1.5708        | 0.529466   | 18.034060


### Funcion Romberg

Este metodo igualmente utiliza los trapecios para calcular el valor de una integral, agregado a esto, se implementa también la extrapolación de Richardson. La función romb() de scipy exige trabajar sobre una "malla" uniforme, es decir, los puntos deben estar equiespaciados y por lo tanto usaremos la función linspace de NumPy para ello. Además, es necesario tener un número de puntos de la malla específico, debemos dar 2^k + 1 puntos para que el método funcione. Esto se debe a que romb va subdividiendo lo intervalos por la mitad hasta alcanzar una buena aproximación como se puede apreciar en la imagen inferior.

<img src="https://image.slidesharecdn.com/metodoromberg-161216192536/75/Metodo-romberg-21-2048.jpg" width="600">

In [10]:
inicio = time.time()
k = 13 #Editable, mientras mas grande mas preciso pero más lento
N = 2**k + 1 #Tamaño malla
x = np.linspace(0, np.pi/2, N) #Malla equiespaciada
dx = x[1] - x[0] #Paso de la malla
romb0 = romb(func0(x), dx=dx)
romb1 = romb(func1(x), dx=dx)
romb2 = romb(func2(x), dx=dx)
romb3 = romb(func3(x), dx=dx)
romb4 = romb(func4(x), dx=dx)
romb5 = romb(func5(x), dx=dx)
fin = time.time()
tiempo_romb = fin - inicio
print("Tiempo calculo en (s)", tiempo_romb)

# Resultados sin error estimado
print("Resultados:")
print("Angulo (rad) | Periodo (s) | Error Relativo (%)")
print(f"{ang[0]:.4f}        | {romb0:.6f}   | {abs((romb0 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[1]:.4f}        | {romb1:.6f}   | {abs((romb1 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[2]:.4f}        | {romb2:.6f}   | {abs((romb2 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[3]:.4f}        | {romb3:.6f}   | {abs((romb3 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[4]:.4f}        | {romb4:.6f}   | {abs((romb4 - T_teorico)/T_teorico)*100:.6f}")
print(f"{ang[5]:.4f}        | {romb5:.6f}   | {abs((romb5 - T_teorico)/T_teorico)*100:.6f}")

Tiempo calculo en (s) 0.008402109146118164
Resultados:
Angulo (rad) | Periodo (s) | Error Relativo (%)
0.2618        | 0.450499   | 0.430058
0.3927        | 0.452932   | 0.972431
0.5236        | 0.456379   | 1.740880
0.7854        | 0.466501   | 3.997334
1.0472        | 0.481397   | 7.318201
1.5708        | 0.529466   | 18.034060


Para hacerse una idea, generemos un gráfico interactivo para visualizar como varía esta función según los valores de L, g y el águlo inicial

In [11]:
# Crear la figura
fig = go.Figure()

# Agregamos los puntos calculados con trapecios
fig.add_trace(go.Scatter(x=ang, y=[trapz0, trapz1, trapz2, trapz3, trapz4, trapz5], mode='markers', name='Trapecios', marker=dict(size=8, symbol='circle')))
fig.add_trace(go.Scatter(x=ang, y=[quad0[0], quad1[0], quad2[0], quad3[0], quad4[0], quad5[0]], mode='markers', name='Quad', marker=dict(size=8, symbol='square')))
fig.add_trace(go.Scatter(x=ang, y=[simp0, simp1, simp2, simp3, simp4, simp5], mode='markers', name='Simpson', marker=dict(size=8, symbol='diamond')))
fig.add_trace(go.Scatter(x=ang, y=[romb0, romb1, romb2, romb3, romb4, romb5], mode='markers', name='Romberg', marker=dict(size=8, symbol='cross')))
fig.add_trace(go.Scatter(x=ang, y=[T_teorico, T_teorico, T_teorico, T_teorico, T_teorico, T_teorico], mode = 'markers', name='Teórico', marker=dict(size=8, symbol='x')))
fig.add_trace(go.Scatter(x=ang, y=exp, mode='markers', name='Experimental', marker=dict(size=10, color='red', symbol='star')))

fig.update_layout(
    title=f"Período de un péndulo vs α₀ (con L={L} m)",
    xaxis_title='Ángulo inicial α₀ [rad]',
    yaxis_title='Período T [s]',
    template='plotly_white'
)

fig.show()
